In [1]:
import os
import sys

sys.path.append("../src")

from dotenv import load_dotenv

from ragas.metrics import LLMContextRecall, Faithfulness, FactualCorrectness

from application.llm_service import LLMService
from infrastructure.text_embedding_pipeline import VectorStore
from application.ragas_evaluator import RAGASEvaluator

load_dotenv("../../secrets.env")
api_key = os.getenv("OPENAI_API_KEY")
file_path = "../data/example.pdf"

/home/sagemaker-user/experiments/ragas_evalutor/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
faiss_vector_store = VectorStore(api_key)
open_ai_llm = LLMService(api_key)

In [3]:
query_answer_pairs = {
    "What is the product?": "GROW",
}
ragas_evaluator = RAGASEvaluator(
    vector_store=faiss_vector_store,
    llm_service=open_ai_llm,
    query_answer_pairs=query_answer_pairs,
)

In [4]:
ragas_evaluator.create_evaluation_dataset(top_k=3)

Loading FAISS index from /home/sagemaker-user/experiments/ragas_evalutor/ragas_evaluation/faiss_index.
Querying FAISS index with query: What is the product?
Retrieved 3 results.
Retrieved 3 relevant documents for query: What is the product?


In [5]:
ragas_evaluator.evaluate(
    metrics=[LLMContextRecall(), Faithfulness(), FactualCorrectness()]
)

Evaluating: 100%|██████████| 3/3 [00:15<00:00,  5.25s/it]


{'context_recall': 1.0000, 'faithfulness': 1.0000, 'factual_correctness(mode=f1)': 0.1700}

In [6]:
ragas_evaluator.evaluation_dataset.samples[0]

SingleTurnSample(user_input='What is the product?', retrieved_contexts=['Grow – Product Summary \nThis product summary is to be read in conjunction with the relevant investment term sheet, product \nhighlights sheet and underlying fund’s prospectus applicable to this product. \n \n1 Introduction', '19 Important Notes \nThis product summary does not form a part of any contract of insurance. It is intended only to be a simplified \ndescription of the product features applicable to this plan and is not exhaustive. Please refer to the terms and \nconditions for more detailed information.', 'https://faq.singlife.com \n \n Start a live chat at our website https://singlife.com \n \n service@singlife.com \nhotline: \n +65 6911 1111 \n \nsinglife.com  Singlife Grow  – Product Summary v3.2         Page 8 of 8'], reference_contexts=None, response='The product is "Grow," which is provided by Singlife. It is an investment plan that has specific features and conditions, and it is accompanied by a re

In [7]:
answers = faiss_vector_store.retrieve_relevant_text(
    "I need to withdraw money from my account, how can I do that?", top_k=5
)

Loading FAISS index from /home/sagemaker-user/experiments/ragas_evalutor/ragas_evaluation/faiss_index.
Querying FAISS index with query: I need to withdraw money from my account, how can I do that?
Retrieved 5 results.


In [8]:
from loguru import logger